In [8]:

import numpy as np
import tensorflow as tf
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from models.CNN import create_cnn_model
import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import json


In [10]:
def load(f):
    # Move up one directory when loading the data
    file_path = os.path.join('../../../', f)
    return np.load(file_path)['arr_0']

# Load the data
X_train = load('kmnist-train-imgs.npz')/ 255.0
x_test = load('kmnist-test-imgs.npz')/ 255.0
Y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')
# Reshape the data for CNN input
X_train = X_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [11]:
# Define the input shape and number of classes
input_shape = X_train.shape[1:]  # 784 for KMNIST
num_classes = Y_train.max() + 1

In [14]:
hyperparameters = {
    'num_layers': 3,
    'filters': [32, 64, 64],
    'kernel_sizes': [(3, 3), (3, 3), (3, 3)],
    'activations': ['relu', 'relu', 'relu'],
    'dense_units': 64,
    'dense_activation': 'relu'
}
configs = [
    {'batch_normalization': False, 'pooling': 'max'},
    {'batch_normalization': True, 'pooling': 'max'},
    {'batch_normalization': False, 'pooling': 'avg'},
    {'batch_normalization': True, 'pooling': 'avg'}
]
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
best_config = None
best_accuracy = 0
histories = []

test_loss_df = pd.DataFrame()
test_accuracy_df = pd.DataFrame()



for config in configs:
    print(f"Testing configuration: {config}")
    hyperparameters.update(config)
    model = create_cnn_model(input_shape, num_classes, hyperparameters)
    history = model.fit(x_train, y_train,
                        epochs=10,
                        batch_size=32,
                        validation_data=(x_val, y_val),
                        verbose=1,
                        callbacks=[early_stopping])
    # Learning curve
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Learning Curve for Config: {config["pooling"]} {config["batch_normalization"]}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    now = datetime.datetime.now()
    plt.savefig(f'learning_curve_{config["pooling"]}_{config["batch_normalization"]}_{now.strftime("%Y-%m-%d")}.png')
    plt.close()
    # Append the last value of val_loss and val_accuracy an additional ten times
    last_val_loss = history.history['val_loss'][-1]
    last_val_accuracy = history.history['val_accuracy'][-1]
    for _ in range(len(history.history['val_loss']), 100):
        history.history['val_loss'].append(last_val_loss)
        history.history['val_accuracy'].append(last_val_accuracy)
    test_loss_df[f"{config['pooling']} {config['batch_normalization']}"] = history.history['val_loss']
    test_accuracy_df[f"{config['pooling']} {config['batch_normalization']}"] = history.history['val_accuracy']


    # Predictions on the validation set
    y_pred = model.predict(x_test)
    y_pred_class = np.argmax(y_pred, axis=1)
    # Confusion matrix
    confusion = confusion_matrix(y_test, y_pred_class)
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion)
    disp.plot()

    plt.title(f'Confusion Matrix for Config: {config["pooling"]} {config["batch_normalization"]}')
    plt.savefig(f'confusion_matrix_{config["pooling"]}_{config["batch_normalization"]}_{now.strftime("%Y-%m-%d")}.png')
    plt.close()
    histories.append(history)
    final_acc = history.history['val_accuracy'][-1]

    if final_acc > best_accuracy:
        best_accuracy = final_acc
        best_config = config
print(f"Best configuration: {best_config}")
print(f"Best accuracy: {best_accuracy}")
# Train the best model on the full training set and evaluate on test set
print("Training best model on full training set...")
hyperparameters = hyperparameters.copy()
hyperparameters.update(best_config)

# Create and compile the best model
best_model = create_cnn_model(input_shape, num_classes, hyperparameters)

# Train on the full training set
history = best_model.fit(x_train, y_train, 
                         epochs=100, 
                         batch_size=16, 
                         validation_data=(x_val, y_val),
                         callbacks=[early_stopping],
                         verbose=1)
last_val_loss = history.history['val_loss'][-1]
last_val_accuracy = history.history['val_accuracy'][-1]
for _ in range(len(history.history['val_loss']), 100):
    history.history['val_loss'].append(last_val_loss)
    history.history['val_accuracy'].append(last_val_accuracy)
# Evaluate on the test set
test_loss, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f'Learning Curve for Best Model: {best_config["pooling"]} {best_config["batch_normalization"]}')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
now = datetime.datetime.now()
plt.savefig(f'learning_curve_best_{best_config["pooling"]}_{best_config["batch_normalization"]}_{now.strftime("%Y-%m-%d")}.png')
plt.close()
# Predictions on the test set
y_pred = best_model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)

# Confusion matrix
confusion = confusion_matrix(y_test, y_pred_class)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion)
disp.plot()
plt.title(f'Confusion Matrix for Best Model: {best_config["pooling"]} {best_config["batch_normalization"]}')
plt.savefig(f'confusion_matrix_best_{best_config["pooling"]}_{best_config["batch_normalization"]}_{now.strftime("%Y-%m-%d")}.png')
plt.close()
# Add the "best + config" column
test_loss_df[f"best {best_config['pooling']} {best_config['batch_normalization']}"] = history.history['val_loss']
test_accuracy_df[f"best {best_config['pooling']} {best_config['batch_normalization']}"] = history.history['val_accuracy']

# Save the dataframes to CSV files
test_loss_df.to_csv('test_loss.csv', index=False)
test_accuracy_df.to_csv('test_accuracy.csv', index=False)





Testing configuration: {'batch_normalization': False, 'pooling': 'max'}
Epoch 1/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.5087 - accuracy: 0.8361 - val_loss: 0.2364 - val_accuracy: 0.9225
Epoch 2/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1852 - accuracy: 0.9436 - val_loss: 0.1487 - val_accuracy: 0.9517
Epoch 3/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1260 - accuracy: 0.9624 - val_loss: 0.1391 - val_accuracy: 0.9530
Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0960 - accuracy: 0.9707 - val_loss: 0.1136 - val_accuracy: 0.9642
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0759 - accuracy: 0.9762 - val_loss: 0.1117 - val_accuracy: 0.9642
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0598 - accuracy: 0.9811 - val_loss: 0.0958 - val_accuracy: 0.9707
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/s

In [37]:
with open('hyperparameters.json', 'w') as f:
    json.dump(hyperparameters, f)

In [38]:
def cutout(image, mask_size=16):
    """Applies Cutout augmentation to an image."""
    h, w, _ = image.shape
    x = np.random.randint(0, w)
    y = np.random.randint(0, h)

    # Ensure the cutout does not go out of bounds
    x1 = np.clip(x - mask_size // 2, 0, w)
    x2 = np.clip(x + mask_size // 2, 0, w)
    y1 = np.clip(y - mask_size // 2, 0, h)
    y2 = np.clip(y + mask_size // 2, 0, h)

    # Create a copy of the image to avoid modifying the original
    image_copy = image.copy()
    image_copy[y1:y2, x1:x2, :] = 0  # Set the cutout region to black

    return image_copy

In [44]:
# Load the hyperparameters
hyperparameters = json.load(open('hyperparameters.json'))
print(hyperparameters)
# Create the models with different noise injection
best_model_rand_noise = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_radial_noise = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_flip = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_zoom = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_mask = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_rand_noise_zoom = create_cnn_model(input_shape, num_classes, hyperparameters)
best_model_no_noise = create_cnn_model(input_shape, num_classes, hyperparameters)

# Random noise
datagen_rand_noise = ImageDataGenerator(
    preprocessing_function=lambda x: np.clip(x + np.random.normal(0, 0.1, x.shape), 0, 1)  # Noise injection
)

# Radial noise
datagen_radial_noise = ImageDataGenerator(
    preprocessing_function=lambda x: np.clip(x + np.random.normal(0, 0.1, x.shape) * np.sqrt(np.sum(np.square(np.indices(x.shape[1:3]).T - np.array(x.shape[1:3])/2), axis=2)).reshape(1, x.shape[1], x.shape[2], 1) / np.max(np.sqrt(np.sum(np.square(np.indices(x.shape[1:3]).T - np.array(x.shape[1:3])/2), axis=2))), 0, 1)  # Radial noise injection
)

# Flip
datagen_flip = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True
)

# Zoom
datagen_zoom = ImageDataGenerator(
    zoom_range=0.1 # Zoom in on the image
)

# Mask
datagen_mask = ImageDataGenerator(
    preprocessing_function = lambda x: cutout(x, mask_size=16) # Zoom in on the image
)
datagen_rand_noise_zoom = ImageDataGenerator(
    preprocessing_function=lambda x: np.clip(x + np.random.normal(0, 0.1, x.shape), 0, 1),  # Noise injection
    zoom_range=0.1 # Zoom in on the image
)
# No noise
datagen_no_noise = ImageDataGenerator()

datagen_val = ImageDataGenerator()
# Wrap the datagen with our custom generator
train_generator_rand_noise = datagen_rand_noise.flow(x_train, y_train, batch_size=32)
train_generator_radial_noise = datagen_radial_noise.flow(x_train, y_train, batch_size=32)
train_generator_flip = datagen_flip.flow(x_train, y_train, batch_size=32)
train_generator_zoom = datagen_zoom.flow(x_train, y_train, batch_size=32)
train_generator_mask = datagen_mask.flow(x_train, y_train, batch_size=32)
train_generator_rand_noise_zoom = datagen_rand_noise_zoom.flow(x_train, y_train, batch_size=32)
train_generator_no_noise = datagen_no_noise.flow(x_train, y_train, batch_size=32)
val_generator = datagen_val.flow(x_val, y_val, batch_size=32)
# Train the model with custom augmentation
augmentation_loss_df = pd.DataFrame()
augmentation_accuracy_df = pd.DataFrame()

# Random noise
history_rand_noise = best_model_rand_noise.fit(train_generator_rand_noise,
                        epochs=15,
                        validation_data=val_generator,  
                        verbose=1
                        )
augmentation_loss_df[f"rand_noise"] = history_rand_noise.history['val_loss']
augmentation_accuracy_df[f"rand_noise"] = history_rand_noise.history['val_accuracy']

# Radial noise
history_radial_noise = best_model_radial_noise.fit(train_generator_radial_noise,
                        epochs=15,
                        validation_data=val_generator,  
                        verbose=1
                        )
augmentation_loss_df[f"radial_noise"] = history_radial_noise.history['val_loss']
augmentation_accuracy_df[f"radial_noise"] = history_radial_noise.history['val_accuracy']


# Flip
history_flip = best_model_flip.fit(train_generator_flip,
                        epochs=15,
                        validation_data=val_generator,
                        verbose=1
                        )
augmentation_loss_df[f"flip"] = history_flip.history['val_loss']
augmentation_accuracy_df[f"flip"] = history_flip.history['val_accuracy']

# Zoom
history_zoom = best_model_zoom.fit(train_generator_zoom,
                        epochs=15,
                        validation_data=val_generator,
                        verbose=1
                        )
augmentation_loss_df[f"zoom"] = history_zoom.history['val_loss']
augmentation_accuracy_df[f"zoom"] = history_zoom.history['val_accuracy']

# Mask
history_mask = best_model_mask.fit(train_generator_mask,
                        epochs=15,
                        validation_data=val_generator,
                        verbose=1
                        )
augmentation_loss_df[f"mask"] = history_mask.history['val_loss']
augmentation_accuracy_df[f"mask"] = history_mask.history['val_accuracy']

# Rand noise and zoom
history_rand_noise_zoom = best_model_rand_noise_zoom.fit(train_generator_rand_noise_zoom,
                        epochs=15,
                        validation_data=val_generator,
                        verbose=1
                        )
augmentation_loss_df[f"rand_noise_zoom"] = history_rand_noise_zoom.history['val_loss']
augmentation_accuracy_df[f"rand_noise_zoom"] = history_rand_noise_zoom.history['val_accuracy']

# No noise
history_no_noise = best_model_no_noise.fit(train_generator_no_noise,
                        epochs=15,
                        validation_data=val_generator,
                        verbose=1
                        )
augmentation_loss_df[f"no_noise"] = history_no_noise.history['val_loss']
augmentation_accuracy_df[f"no_noise"] = history_no_noise.history['val_accuracy']

augmentation_loss_df.to_csv('augmentation_loss.csv', index=False)
augmentation_accuracy_df.to_csv('augmentation_accuracy.csv', index=False)



{'num_layers': 3, 'filters': [32, 64, 64], 'kernel_sizes': [[3, 3], [3, 3], [3, 3]], 'activations': ['relu', 'relu', 'relu'], 'dense_units': 64, 'dense_activation': 'relu', 'batch_normalization': True, 'pooling': 'avg'}
Epoch 1/15
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1963 - accuracy: 0.9473 - val_loss: 0.0956 - val_accuracy: 0.9685
Epoch 2/15
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0700 - accuracy: 0.9792 - val_loss: 0.0603 - val_accuracy: 0.9827
Epoch 3/15
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0469 - accuracy: 0.9859 - val_loss: 0.0517 - val_accuracy: 0.9817
Epoch 4/15
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0405 - accuracy: 0.9873 - val_loss: 0.0437 - val_accuracy: 0.9868
Epoch 5/15
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0312 - accuracy: 0.9905 - val_loss: 0.0539 - val_accuracy: 0.9830
Epoch 6/15
1688/1688 [==============================] - 9s 5

In [45]:
rand_noise_loss, rand_noise_accuracy = best_model_rand_noise.evaluate(x_test, y_test, verbose=0)
print(f"Rand noise test accuracy: {rand_noise_accuracy:.4f}")
print(f"Rand noise test loss: {rand_noise_loss:.4f}")
radial_noise_loss, radial_noise_accuracy = best_model_radial_noise.evaluate(x_test, y_test, verbose=0)
print(f"Radial noise test accuracy: {radial_noise_accuracy:.4f}")
print(f"Radial noise test loss: {radial_noise_loss:.4f}")
flip_loss, flip_accuracy = best_model_flip.evaluate(x_test, y_test, verbose=0)
print(f"Flip test accuracy: {flip_accuracy:.4f}")
print(f"Flip test loss: {flip_loss:.4f}")
zoom_loss, zoom_accuracy = best_model_zoom.evaluate(x_test, y_test, verbose=0)
print(f"Zoom test accuracy: {zoom_accuracy:.4f}")
print(f"Zoom test loss: {zoom_loss:.4f}")
mask_loss, mask_accuracy = best_model_mask.evaluate(x_test, y_test, verbose=0)
print(f"Mask test accuracy: {mask_accuracy:.4f}")
print(f"Mask test loss: {mask_loss:.4f}")
rand_noise_zoom_loss, rand_noise_zoom_accuracy = best_model_rand_noise_zoom.evaluate(x_test, y_test, verbose=0)
print(f"Rand noise and zoom test accuracy: {rand_noise_zoom_accuracy:.4f}")
print(f"Rand noise and zoom test loss: {rand_noise_zoom_loss:.4f}")
no_noise_loss, no_noise_accuracy = best_model_no_noise.evaluate(x_test, y_test, verbose=0)
print(f"No noise test accuracy: {no_noise_accuracy:.4f}")
print(f"No noise test loss: {no_noise_loss:.4f}")


Rand noise test accuracy: 0.9705
Rand noise test loss: 0.1497
Radial noise test accuracy: 0.9721
Radial noise test loss: 0.1365
Flip test accuracy: 0.9229
Flip test loss: 0.2834
Zoom test accuracy: 0.9672
Zoom test loss: 0.1553
Mask test accuracy: 0.9626
Mask test loss: 0.1412
Rand noise and zoom test accuracy: 0.9667
Rand noise and zoom test loss: 0.1692
No noise test accuracy: 0.9665
No noise test loss: 0.1730


In [46]:
# List of models and their names
models = [
    (best_model_rand_noise, "Random Noise"),
    (best_model_radial_noise, "Radial Noise"),
    (best_model_flip, "Flip"),
    (best_model_zoom, "Zoom"),
    (best_model_mask, "Mask"),
    (best_model_rand_noise_zoom, "Random Noise + Zoom"),
    (best_model_no_noise, "No Noise")
]

# Create and save confusion matrices for each model
for model, name in models:
    # Predictions on the test set
    y_pred = model.predict(x_test)
    y_pred_class = np.argmax(y_pred, axis=1)

    # Confusion matrix
    confusion = confusion_matrix(y_test, y_pred_class)
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion)
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    disp.plot(ax=ax)
    plt.title(f'Confusion Matrix for {name}')
    
    # Save the figure
    now = datetime.datetime.now()
    plt.savefig(f'confusion_matrix_{name.replace(" ", "_").lower()}_{now.strftime("%Y-%m-%d")}.png')
    plt.close()




313/313 [==============================] - 1s 2ms/step
